Importing relavent Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

Load the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
raw_csv = np.loadtxt("/content/drive/My Drive/Colab Notebooks/Audiobooks_data.csv", delimiter=',', skiprows=1)

In [ ]:
raw_csv

array([[9.9400e+02, 1.6200e+03, 1.6200e+03, ..., 5.0000e+00, 9.2000e+01,
        0.0000e+00],
       [1.1430e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.0590e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 3.8800e+02,
        0.0000e+00],
       ...,
       [3.1134e+04, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.2832e+04, 1.6200e+03, 1.6200e+03, ..., 0.0000e+00, 9.0000e+01,
        0.0000e+00],
       [2.5100e+02, 1.6740e+03, 3.3480e+03, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

In [ ]:
unscaled_raw_input = raw_csv[:, 1:-1]
unscaled_target = raw_csv[:,-1]

Balance the Dataset

In [ ]:
num_one_targets = int(np.sum(unscaled_target))
zero_target_counter = 0
indices_to_remove = []

for i in range(unscaled_target.shape[0]):
  if unscaled_target[i]==0:
    zero_target_counter += 1
    if zero_target_counter > num_one_targets:
      indices_to_remove.append(i)

In [ ]:
unscaled_input_priors = np.delete(unscaled_raw_input, indices_to_remove, axis=0)
unscaled_target_priors = np.delete(unscaled_target, indices_to_remove, axis=0)

Scaling the Dataset

In [ ]:
scaled_inputs = preprocessing.scale(unscaled_input_priors)

Shuffled Indices

In [ ]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

In [ ]:
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = unscaled_target_priors[shuffled_indices]

In [ ]:
shuffled_indices

array([ 821,  605, 1553, ...,  225, 1140, 3657])

In [ ]:
samples_count = shuffled_inputs.shape[0]

In [ ]:
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_cout = samples_count - train_samples_count - validation_samples_count

In [ ]:
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

Print the size of train, test, validation

In [ ]:
print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_cout, np.sum(test_targets)/test_samples_cout)

1798.0 3579 0.502374965074043
219.0 447 0.4899328859060403
220.0 448 0.49107142857142855


Save the data in Numpy format

In [ ]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

Loading the data

In [ ]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

In [ ]:
npz = np.load('Audiobooks_data_validation.npz')

validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

In [ ]:
npz = np.load('Audiobooks_data_test.npz')

test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

Model

In [ ]:
input_size = 10
hidden_layer_size = 50
output_size = 2

In [ ]:
model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)

model.fit(train_inputs, train_targets, epochs=100, batch_size=100, 
          validation_data=(validation_inputs, validation_targets), callbacks=[early_stopping])

Epoch 1/100
36/36 [==============================] - 0s 3ms/step - loss: 0.3208 - accuracy: 0.8256 - val_loss: 0.3404 - val_accuracy: 0.8277
Epoch 2/100
36/36 [==============================] - 0s 3ms/step - loss: 0.3215 - accuracy: 0.8296 - val_loss: 0.3390 - val_accuracy: 0.8255
Epoch 3/100
36/36 [==============================] - 0s 3ms/step - loss: 0.3190 - accuracy: 0.8262 - val_loss: 0.3511 - val_accuracy: 0.8255
Epoch 4/100
36/36 [==============================] - 0s 3ms/step - loss: 0.3240 - accuracy: 0.8254 - val_loss: 0.3386 - val_accuracy: 0.8121
Epoch 5/100
36/36 [==============================] - 0s 3ms/step - loss: 0.3164 - accuracy: 0.8296 - val_loss: 0.3469 - val_accuracy: 0.8210
Epoch 6/100
36/36 [==============================] - 0s 3ms/step - loss: 0.3162 - accuracy: 0.8318 - val_loss: 0.3413 - val_accuracy: 0.8121
Epoch 7/100
36/36 [==============================] - 0s 3ms/step - loss: 0.3141 - accuracy: 0.8293 - val_loss: 0.3420 - val_accuracy: 0.8233


Test the model

In [ ]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 1ms/step - loss: 0.3857 - accuracy: 0.7969


In [ ]:
print('Test loss: {0:.2f}\nTest accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.39
Test accuracy: 79.69%
